In [1]:
# On récupère les dataframes d'entraînement et de tests stockés en mémoire
%store -r df_train
%store -r df_test

# On sépare les données de leur classe
x_train = df_train.drop(columns=['species'])
t_train = df_train['species']

In [2]:
# On récupère les meilleurs hyperparamètres trouvés pour chaque méthode

# Perceptron
%store -r perceptron_best_penalty
%store -r perceptron_best_learning_rate
%store -r perceptron_best_alpha

# Neural Network
%store -r neural_network_best_alpha

# K Neighbors
%store -r k_neighbors_best_n
%store -r k_neighbors_best_p

# Support Vector
%store -r support_vector_best_c

# Decision Tree
%store -r decision_tree_best_criterion
%store -r decision_tree_best_ccp_alpha

# Ada Boost
%store -r ada_boost_best_n_estimators
%store -r ada_boost_best_learning_rate

In [3]:
# On applique chaque entraînement sur le dataset d'entraînement

# Perceptron
%run ../methodes/perceptron_definition.ipynb import Perceptron_
perceptron = Perceptron_(perceptron_best_penalty, perceptron_best_learning_rate, perceptron_best_alpha)
perceptron.train(x_train, t_train)

# Neural Network
%run ../methodes/neural_network_definition.ipynb import Neural_Network_
neural_network = Neural_Network_(neural_network_best_alpha)
neural_network.train(x_train, t_train)

# K Neighbors
%run ../methodes/k_neighbors_definition.ipynb import K_Neighbors_
k_neighbors = K_Neighbors_(k_neighbors_best_n, k_neighbors_best_p)
k_neighbors.train(x_train, t_train)

# Support Vector
%run ../methodes/support_vector_definition.ipynb import Support_Vector_
support_vector = Support_Vector_(support_vector_best_c)
support_vector.train(x_train, t_train)

# Decision Tree
%run ../methodes/decision_tree_definition.ipynb import Decision_Tree_
decision_tree = Decision_Tree_(decision_tree_best_criterion, decision_tree_best_ccp_alpha)
decision_tree.train(x_train, t_train)

# Ada Boost
%run ../methodes/ada_boost_definition.ipynb import Ada_Boost_
ada_boost = Ada_Boost_(ada_boost_best_n_estimators, ada_boost_best_learning_rate)
ada_boost.train(x_train, t_train)

C:\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [4]:
# On effectue la prédiction avec chaque méthode sur l'ensemble de test

# Perceptron
perceptron_pred = perceptron.predict(df_test)

# Neural Network
neural_network_pred = neural_network.predict(df_test)

# K Neighbors
k_neighbors_pred = k_neighbors.predict(df_test)

# Support Vector
support_vector_pred = support_vector.predict(df_test)

# Decision Tree
decision_tree_pred = decision_tree.predict(df_test)

# Ada Boost
ada_boost_pred = ada_boost.predict(df_test)

In [5]:
# Création d'un dataframe qui contient les prédictions de toutes les méthodes
import pandas as pd
all_pred_df = pd.DataFrame([perceptron_pred, neural_network_pred, k_neighbors_pred, support_vector_pred, 
                            decision_tree_pred, ada_boost_pred], index = ['perceptron', 'neural network',
                                                                           'k_neighbors', 'support vector',
                                                                           'decision tree', 'ada boost'])
all_pred_df = all_pred_df.transpose()

# On crée une colonne qui contient les prédictions les plus présentes pour chaque donnée
all_pred_df['general'] = all_pred_df.mode(axis=1)[0]

In [6]:
# On crée une matrice de confusion entre chaque méthode
# L'élément i,j représente le nombre de fois moyen que les méthodes i et j on prédit la même classe
# On prend également en compte la colonne 'general'

confusion_matrix = pd.DataFrame(0, index=all_pred_df.columns, columns=all_pred_df.columns)
for index in confusion_matrix.index:
    for column in confusion_matrix.columns:
        confusion_matrix[column][index] = np.where(all_pred_df[column] == all_pred_df[index])[0].shape[0]
confusion_matrix /= all_pred_df.shape[0] 

In [7]:
# Affichage des résultats
pd.options.display.float_format = '{:,.2f}'.format
confusion_matrix

,perceptron,neural network,k_neighbors,support vector,decision tree,ada boost,general
perceptron,1.00,0.93,0.93,0.92,0.55,0.77,0.93
neural network,0.93,1.00,0.99,0.99,0.57,0.81,0.99
k_neighbors,0.93,0.99,1.00,0.99,0.57,0.81,0.99
support vector,0.92,0.99,0.99,1.00,0.56,0.82,0.99
decision tree,0.55,0.57,0.57,0.56,1.00,0.52,0.57
ada boost,0.77,0.81,0.81,0.82,0.52,1.00,0.81
general,0.93,0.99,0.99,0.99,0.57,0.81,1.00
